Performing multitask learning on the COCO dataset to perform both object detection and segmentation using TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

Define the input shape for the images

In [ ]:

input_shape = (224, 224, 3)


Define the number of classes for object detection

In [ ]:

num_classes_obj_det = 80

Define the number of classes for segmentation

In [ ]:

num_classes_segmentation = 1

Define the ResNet50 backbone for feature extraction

In [ ]:

backbone = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

Define the object detection head

In [ ]:

x = backbone.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
object_detection_output = Dense(num_classes_obj_det, activation='softmax', name='object_detection_output')(x)

Define the segmentation head

In [ ]:
x = backbone.output
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(num_classes_segmentation, (1, 1), activation='sigmoid', name='segmentation_output')(x)

Define the model

In [ ]:

model = Model(inputs=backbone.input, outputs=[object_detection_output, segmentation_output])

Compile the model

In [ ]:

model.compile(optimizer='adam', loss={'object_detection_output': 'categorical_crossentropy', 'segmentation_output': 'binary_crossentropy'})

Load the COCO dataset for multitask learning

In [ ]:

(train_images, train_labels_obj_det, train_labels_segmentation), (test_images, test_labels_obj_det, test_labels_segmentation) = keras.datasets.coco.load_data()

Preprocess the input images for object detection

In [ ]:

train_images_obj_det = keras.applications.resnet50.preprocess_input(train_images)
test_images_obj_det = keras.applications.resnet50.preprocess_input(test_images)

Preprocess the input images for segmentation

In [ ]:

train_images_segmentation = train_images / 255.0
test_images_segmentation = test_images / 255.0


Convert the labels to one-hot encoding for object detection

In [ ]:

train_labels_obj_det_one_hot = keras.utils.to_categorical(train_labels_obj_det, num_classes_obj_det)
test_labels_obj_det_one_hot = keras.utils.to_categorical(test_labels_obj_det, num_classes_obj_det)

Reshape the labels for segmentation

In [ ]:

train_labels_segmentation = train_labels_segmentation.reshape((-1, 224, 224, num_classes_segmentation))
test_labels_segmentation = test_labels_segmentation.reshape((-1, 224, 224, num_classes_segmentation))

Train the model

In [ ]:

model.fit(train_images, {'object_detection_output': train_labels_obj_det_one_hot, 'segmentation_output': train_labels_segmentation}, epochs=10, batch_size=32, validation_data=(test_images, {'object_detection_output': test_labels_obj_det_one_hot, 'segmentation_output': test_labels_segmentation}))

Evaluate the model

In [ ]:

test_loss, test_accuracy = model.evaluate(test_images, {'object_detection_output': test_labels_obj_det_one_hot, 'segmentation_output': test_labels_segmentation})

#Print accuracy
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


Compile the model with multiple losses

In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss={'box_output': YOLOLoss(),
                    'seg_output': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)},
              loss_weights={'box_output': 1.0, 'seg_output': 0.5})

Train the model

In [ ]:

model.fit(train_dataset, epochs=10, validation_data=val_dataset)

mean average precision (mAP) used to evaluate the performance of object detection models

mean intersection over union (mIoU) are used evaluate the performance of segmentation models

In [ ]:
# Calculate mAP
mAP = tfa.metrics.mAP(num_classes=num_classes)

# Calculate mIoU
mIoU = tf.keras.metrics.MeanIoU(num_classes=num_classes)


Print the mAP and mIoU

In [ ]:

print('mAP:', results[1])
print('mIoU:', results[2])

Evaluate the model on test data

In [ ]:

results = model.evaluate(test_dataset)

Save the model

In [ ]:

model.save('multitask_coco.h5')